In [98]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os

import numpy as np
from scipy.signal import butter, sosfiltfilt, find_peaks

# --- 1. Load ECG Data with HR Labels ---

In [101]:
file_path = os.path.join("data", "ecg_data_with_hr_labels.pkl")  

# Load ECG data
with open(file_path, "rb") as f:
    data = pickle.load(f)
    
# Extract signals and ground truth HR values
signals = data["signals"]
ground_truth_hr = data["hr_values"]

print(f"Loaded {len(signals)} ECG signals.")

Loaded 200 ECG signals.


# --- 2. Implement Your HR Extraction Algorithm Here ---

**Instruction:**

***Your algorithm should return a list of HR values where each HR value corresponds to an ECG signal. Ensure the length of the list is 200 (equal to number of signals). The list can include np.nan if your algorithm is not able to calculate HR for a signal.***

In [104]:
def peaks(signalz, order=6, cutoff=45, fs=200):
    sos = butter(order, cutoff, fs=fs, btype='low', output='sos')
    filtered_signal = sosfiltfilt(sos, signalz)
    peaks, _ = find_peaks(filtered_signal, height=(0.4*max(filtered_signal)), distance=0.35*fs)
    return peaks

def compute_rr_intervals(r_peaks, fs=200):
    rr_intervals = np.diff(r_peaks) / fs
    return rr_intervals

def HR(rri):
    return 60/(np.mean(rri))


def extract_HR(ecg_signals):
    detected_hr_values = [np.nan] * 200
    for i, signal in enumerate(ecg_signals):
        peaks_ = peaks(signal)
        rri = compute_rr_intervals(peaks_)
        hr = HR(rri)
        detected_hr_values[i] = hr
    return detected_hr_values

detected_hr_values = extract_HR(signals)

# --- 3. Evaluation of HR Extraction Algorithm ---

**Description:**

***Evaluates the performance of your HR extraction method using Mean Absolute Error (MAE).***

In [107]:
def evaluate_hr_extraction(detected_hr_values, ground_truth_hr):
    """
    Evaluates HR extraction performance using Mean Absolute Error (MAE).
    
    Parameters:
        - detected_hr_values (List of floats): List of detected HR values.
        - ground_truth_hr (List of floats): List of ground truth HR values.
    
    Returns:
        - MAE score.
    """
    valid_indices = np.where(~np.isnan(detected_hr_values))[0].tolist() 
    
    if len(valid_indices) == 0:
        return {"Mean Absolute Error": np.nan}
    
    absolute_errors = np.abs(np.array(ground_truth_hr)[valid_indices] - np.array(detected_hr_values)[valid_indices])
    mae = np.sum(absolute_errors) / len(absolute_errors)
    return {"Mean Absolute Error": mae}

In [109]:
# Evaluate performance of your HR extraction method
mae = evaluate_hr_extraction(detected_hr_values, ground_truth_hr)

print(mae)

{'Mean Absolute Error': 8.434749199806124}
